# Convert .reg files to hipscat

## Import libraries

In [1]:
from multiprocessing import cpu_count
from pathlib import Path

import cdshealpix
import dask.distributed
import hipscat.io.file_io
import numpy as np
import lsdb
import mom_builder
import pyarrow as pa
from astropy.coordinates import Angle, Latitude, Longitude
from astropy.table import Table
from hipscat.pixel_math.hipscat_id import HIPSCAT_ID_COLUMN
from hipscat_import.catalog.arguments import ImportArguments as HipscatImportArguments
from hipscat_import.pipeline import pipeline as hipscat_import_pipeline
from mocpy import MOC
from tqdm import tqdm

## Constants

In [2]:
HIGHEST_ORDER = 14  # Smallest tiles to start with
SPLIT_ORDER = 5  # Split building tasks into tiles of this order
DEPTH_DELTA = 2  # Specifies the precision of the healpix filtering, higher is better, lower is faster
MOCPY_THREADS = None  # Number of threads to use with mocpy
MOM_BUILDER_THREADS = cpu_count()  # Number of threads to use with mom_builder
DASK_WORKERS = cpu_count()  # Number of dask workers, for hipscat-import

In [3]:
VALUE_NP_DTYPE = np.dtype(np.uint8)  # dtype object is required by mom_builder
VALUE_PA_TYPE = pa.from_numpy_dtype(VALUE_NP_DTYPE)
VALUE_COLUMN = 'value'  # Name of the value column in the output Hipscat

In [4]:
REGION_DIR = Path('.')
BOX_REGIONS = REGION_DIR / 'box.reg.I'
DIP_REGIONS = REGION_DIR / 'dip.reg.I.nodups_head'
STARS_REGIONS = REGION_DIR / 'stars.reg.I.nodups_head'

POINTS_MAP_FITS = Path('./hsc-pdr3-forced-r_points_map.fits')

MOM_DIR = Path('./mom_parquet')
HIPSCAT_DIR = Path('./hipscat')
HIPSCAT_NAME = 'hsc_pdr3_mask_i'  # bright star masks for HSC PDR3 I-band

## Read ascii region files and convert to healpix tiles of `HIGHEST_ORDER`

In [5]:
def read_circles_cdshealpix(filename):
    table = Table.read(
        filename,
        format='ascii',
        names=['ra', 'dec', 'radius'],
        units=['deg'] * 3,
    )
    ra = Longitude(table['ra'])
    dec = Latitude(table['dec'])
    radius = Angle(table['radius'])
    
    hp_index = []
    for lon, lat, r in zip(ra, dec, radius):
        idx, _hp_depth, _fully_covered = cdshealpix.cone_search(
            lon,
            lat,
            r,
            depth=HIGHEST_ORDER,
            depth_delta=DEPTH_DELTA,
            flat=True
        )
        hp_index.extend(idx)
        
    return np.array(hp_index, dtype=np.uint64)


def read_circles_mocpy(filename):
    table = Table.read(
        filename,
        format='ascii',
        names=['ra', 'dec', 'radius'],
        units=['deg'] * 3,
    )
    ra = Longitude(table['ra'])
    dec = Latitude(table['dec'])
    radius = Angle(table['radius'])
    
    mocs = MOC.from_cones(
        lon=ra,
        lat=dec,
        radius=radius,
        max_depth=HIGHEST_ORDER,
        delta_depth=DEPTH_DELTA,
        n_threads=MOCPY_THREADS,
    )
    hp_index = np.concatenate([moc.flatten() for moc in mocs])
    return hp_index


# mocpy is faster, even for a single thread
read_circles = read_circles_mocpy


# We use mocpy here because cdshealpix doesn't support box searches
def read_boxes(filename):
    table = Table.read(
        filename,
        format='ascii',
        names=['ra_c', 'dec_c', 'width', 'height'],
        units=['deg'] * 4
    )
    ra_center = Longitude(table['ra_c'])
    dec_center = Latitude(table['dec_c'])
    width = Angle(table['width'])
    height = Angle(table['height'])
    
    width_larger = width > height
    a = np.where(width_larger, 0.5 * width, 0.5 * height)
    b = np.where(width_larger, 0.5 * height, 0.5 * width)
    angle = np.where(width_larger, Angle(90, 'deg'), 0)
    
    mocs = MOC.from_boxes(
        lon=ra_center,
        lat=dec_center,
        a=a,
        b=b,
        angle=angle,
        max_depth=HIGHEST_ORDER,
        n_threads=MOCPY_THREADS,
    )
    hp_index = np.concatenate([moc.flatten() for moc in mocs])
    return hp_index

In [6]:
%time dip_index = read_circles(DIP_REGIONS)
%time stars_index = read_circles(STARS_REGIONS)
%time box_index = read_boxes(BOX_REGIONS)

CPU times: user 3.58 s, sys: 69.5 ms, total: 3.65 s
Wall time: 1.33 s
CPU times: user 1.7 s, sys: 54.3 ms, total: 1.76 s
Wall time: 1.12 s
CPU times: user 10.7 s, sys: 159 ms, total: 10.8 s
Wall time: 2.67 s


### Group masks by pixels of `SPLIT_ORDER`

In [7]:
def parent(index, child_order, parent_order):
    delta_order = np.array(child_order - parent_order, dtype=np.uint64)
    assert np.all(delta_order >= 0), 'parent_order must not be lower than the child_order'
    # For some numpyish reason, we cannot just do >>
    shift_bits = np.array(2, dtype=np.uint64) * delta_order
    return np.right_shift(np.array(index, dtype=np.uint64), shift_bits)


def first_child(index, parent_order, child_order):
    delta_order = np.array(child_order - parent_order, dtype=np.uint64)
    assert np.all(delta_order >= 0), 'parent_order must not be lower than the child_order'
    # For some numpyish reason, we cannot just do <<
    shift_bits = np.array(2, dtype=np.uint64) * delta_order
    return np.left_shift(np.array(index, dtype=np.uint64), shift_bits)


def group_by_order(hp_index, hp_order, split_order):
    # Sort and get unique indexes
    hp_index = np.unique(hp_index)
    
    split_order_npix = 12 * (4 ** split_order)
    split_order_indexes = np.arange(1, split_order_npix, dtype=np.uint64)
    hp_order_separation_indexes = first_child(split_order_indexes, split_order, hp_order)
    
    # Split input indexes by the location in split_order tiles
    split_tile_boundaries = np.searchsorted(hp_index, hp_order_separation_indexes)
    lookup = np.split(hp_index,split_tile_boundaries)
    
    return lookup

In [8]:
%%time

dip_lookup = group_by_order(dip_index, HIGHEST_ORDER, SPLIT_ORDER)
del dip_index
stars_lookup = group_by_order(stars_index, HIGHEST_ORDER, SPLIT_ORDER)
del stars_index
box_lookup = group_by_order(box_index, HIGHEST_ORDER, SPLIT_ORDER)
del box_index

CPU times: user 564 ms, sys: 36 ms, total: 600 ms
Wall time: 609 ms


### Read coverage map (Hipscat's `points_map.fits`) into MOC

In [9]:
coverage = hipscat.io.file_io.read_fits_image(POINTS_MAP_FITS)
# Convert coverage to bool and then convert to the target type
coverage = np.asarray(coverage > 0, dtype=VALUE_NP_DTYPE)

# next I bascially copy HealpixDataset._read_moc_from_point_map()
# https://github.com/astronomy-commons/hipscat/blob/04596506011ec5f021ebe612dcf7f5e8e9044795/src/hipscat/catalog/healpix_dataset/healpix_dataset.py#L114

# length of the file is 12 * 4**order
coverage_order_float = 0.5 * np.log2(coverage.shape[0] / 12)
coverage_order = int(coverage_order_float)
assert coverage_order == coverage_order_float, 'coverage_order must be an integer'
assert coverage_order <= HIGHEST_ORDER, 'coverage_order is larger than HIGHEST_ORDER, cast coverage map to HIGHEST_ORDER'

## Build Multi-Order Continious Map (MOM)

### Define a function to get `HIGHEST_ORDER` tile values

In [10]:
def get_value(order, indexes):
    assert order == HIGHEST_ORDER
    
    first_index = indexes[0]
    top_index = parent(first_index, HIGHEST_ORDER, SPLIT_ORDER)
    indexes_coverage_order = parent(first_index, HIGHEST_ORDER, coverage_order)

    # Get HIGHEST_ORDER indexes and shift them to start from 0
    dip = dip_lookup[top_index] - first_index
    stars = stars_lookup[top_index] - first_index
    box = box_lookup[top_index] - first_index
    
    values = np.zeros(indexes.shape, dtype=VALUE_NP_DTYPE)
    
    # coverage is the first bit, coverage array is either 0 or 1
    values[:] |= coverage[indexes_coverage_order]
    # Masks arrays can be safely shifted by the smallest index value
    # dip is the second bit
    values[dip] |= 1 << 2
    # stars and their boxes is the third bit
    values[stars] |= 1 << 3
    values[box] |= 1 << 3
    
    return values

### Define a MOM merging strategy

In [11]:
mom_merger = mom_builder.MOMMerger(
    # Each healpix tree leaf would have a single value
    # It is a coincidence that we have a column named 'value' 
    state='value',
    # If four children have the same value, they are merged into a single parent
    merger='equal',
    dtype=VALUE_NP_DTYPE,
)

### Initialize parquet writer object

In [12]:
from writer import Writer

writer = Writer(
    MOM_DIR,
    col_name=VALUE_COLUMN,
    col_type=VALUE_PA_TYPE,
)

### Run MOM building pipeline

In [13]:
%%time

generator = mom_builder.gen_mom_from_fn(
    fn=get_value,
    max_norder=HIGHEST_ORDER,
    split_norder=SPLIT_ORDER,
    merger=mom_merger,
    n_threads=MOM_BUILDER_THREADS,
    tiles_consumer=writer.write,  # immediately write tiles to parquet, output nothing from the generator
)

# Generator yields nothing, because tiles_consumer returns nothing
for _ in tqdm(generator, total=12 * 4**SPLIT_ORDER + 1):
    pass

del dip_lookup, stars_lookup, box_lookup

100%|██████████| 12289/12289 [00:12<00:00, 970.84it/s] 

CPU times: user 1min 25s, sys: 7.49 s, total: 1min 32s
Wall time: 12.7 s


## Create Hipscat with `hipscat-import`

In [14]:
%%time

hipscat_import_args = HipscatImportArguments(
    input_path=MOM_DIR,
    file_reader='parquet',
    output_path=HIPSCAT_DIR,
    output_artifact_name=HIPSCAT_NAME,
    # We have _hipscat_index predefined, it points to the "begging" of a tile
    # (smallest index of order 19 children)
    use_hipscat_index=True,
    add_hipscat_index=False,
) 

with dask.distributed.Client(n_workers=DASK_WORKERS) as client:
    display(client)
    hipscat_import_pipeline(hipscat_import_args)

Planning  :   0%|          | 0/5 [00:00<?, ?it/s]

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 12,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50713,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:50740,Total threads: 1
Dashboard: http://127.0.0.1:50742/status,Memory: 2.67 GiB
Nanny: tcp://127.0.0.1:50716,


/Users/hombit/.virtualenvs/elzorro/lib/python3.12/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50778 instead
  warnings.warn(


Mapping   :   0%|          | 0/12289 [00:00<?, ?it/s]

Binning   :   0%|          | 0/2 [00:00<?, ?it/s]

Splitting :   0%|          | 0/12289 [00:00<?, ?it/s]

Reducing  :   0%|          | 0/57 [00:00<?, ?it/s]

Finishing :   0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 39.9 s, sys: 15.1 s, total: 55 s
Wall time: 2min 37s


### Load catalog with LSDB

In [16]:
catalog = lsdb.read_hipscat(HIPSCAT_DIR / HIPSCAT_NAME)
with dask.distributed.Client(n_workers=DASK_WORKERS) as client:
    display(client)
    df = catalog.query("value == 13").head(100)
display(df)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 12,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50906,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:50938,Total threads: 1
Dashboard: http://127.0.0.1:50946/status,Memory: 2.67 GiB
Nanny: tcp://127.0.0.1:50909,


,pixel_Norder,pixel_Npix,value,Norder,Dir,Npix
_hipscat_index,,,,,,
1154599350760898560,14,268826110,13,0,0,1
1154599355055865856,14,268826111,13,0,0,1
1154600763805138944,14,268826439,13,0,0,1
1154600772395073536,14,268826441,13,0,0,1
1154600780985008128,14,268826443,13,0,0,1
...,...,...,...,...,...,...
1154882990401126400,14,268892150,13,0,0,1
1154882994696093696,14,268892151,13,0,0,1
1154883016170930176,13,67223039,13,0,0,1
